## IMDB 영화평 감성분석 - 정확도 높이기

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
# <br />태그는 공백으로 변환
df['review'] = df.review.str.replace('<br />',' ')

# 구둣점, 숫자 제거 --> 영어 이외의 문자는 공백으로 변환
import re
df['review'] = df.review.apply(lambda x: re.sub('[^A-Za-z]', ' ', x))

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2021
)

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english', ngram_range=(1,3))
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [ ]:
# cvect.vocabulary_

### 1. LogisticRegression

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 300)
%time lr.fit(X_train_cv, y_train)

Wall time: 1min 2s


LogisticRegression(max_iter=300)

In [21]:
from sklearn.metrics import accuracy_score
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.88784

### 2. DecisionTree

In [22]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor()
%time dt.fit(X_train_cv, y_train)

Wall time: 5min 6s


DecisionTreeRegressor()

In [23]:
pred = dt.predict(X_test_cv)
accuracy_score(y_test, pred)

0.73008

### 3. SVM

In [24]:
from sklearn.svm import SVC
svc = SVC()
%time svc.fit(X_train_cv, y_train)

Wall time: 19min 57s


SVC()

In [25]:
pred = svc.predict(X_test_cv)
accuracy_score(y_test, pred)

0.87408

### 4. RandomForest

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
%time rf.fit(X_train_cv, y_train)

Wall time: 8min 38s


RandomForestClassifier()

In [27]:
pred = rf.predict(X_test_cv)
accuracy_score(y_test, pred)

0.84448

### 5. 앙상블

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
lr = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()

In [29]:
from sklearn.ensemble import VotingClassifier
vo_clf = VotingClassifier(
    estimators = [('LR', lr), ('SVC', svc), ('KNN', knn)],
    voting = 'hard'
)

In [30]:
vo_clf.fit(X_train_cv, y_train)
pred = vo_clf.predict(X_test_cv)
accuracy_score(y_test, pred)

C:\Users\kimee\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.88192